# Goals and Overview

The goal of this project is to develop a model that predicts whether a customer will leave the bank soon. After taking a first look at the data and fixing missing and duplicate values, I will begin with feature preparation in which I will standardize all data with StandartScaler() and One-Hot Encoding. Different models will be explored and and classes will be balanced to improve model f1 score, and find the best model.

Beta Bank customers are leaving: little by little, chipping away every month. The bankers figured out it’s cheaper to save the existing customers rather than to attract new ones.

We need to predict whether a customer will leave the bank soon. You have the data on clients’ past behavior and termination of contracts with the bank.

Build a model with the maximum possible F1 score. To pass the project, you need an F1 score of at least 0.59. Check the F1 for the test set.

Additionally, measure the AUC-ROC metric and compare it with the F1.

# Project

## Initialization

In [ ]:
#Loading necessary libraries.
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  accuracy_score, precision_recall_curve, roc_auc_score, confusion_matrix, f1_score, make_scorer, recall_score, precision_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np

## Reading Data

In [ ]:
#Reading Data.
data = pd.read_csv('/datasets/Churn.csv')

In [ ]:
#Looking at 'data'.
data

In [ ]:
data[data['Exited'] == 0]

In [ ]:
data[data['Exited'] == 1]

In [ ]:
data['Geography'].value_counts()

In [ ]:
data['CreditScore'].value_counts()

In [ ]:
data['Age'].value_counts()

In [ ]:
data.info()

All datatypes seem to be correct, except Tenure which seems like it could be 'int64'. Column names are fine as is.

__Missing Values__

In [ ]:
#Checking for missing values.
data.isna().sum()

The only column with missing values is 'Tenure', which will be treated as having 0 years of Tenure. This is becuase the other features are still valuable in the analysis.

__Duplicate Values__

In [ ]:
#Chekcing for duplicate values.
data[data.duplicated()]

There are no duplicated rows.

## Data Preparation

In [ ]:
data

In [ ]:
#Dropping 'Surname', 'CustomerId', and 'RowNumber' from 'data'.
data.drop(columns=['Surname', 'CustomerId', 'RowNumber'], inplace=True)
data.drop(columns=['CustomerId'], inplace=True)
data.drop(columns=['RowNumber'], inplace=True)

In [ ]:
#Looking at unique values in 'Geography'.
data['Geography'].unique()

In [ ]:
#Filling missing values in 'Tenure' with 0.
data.fillna(0, inplace=True)

Columns have been dropped as the data like 'RowNumber', 'Surname', and 'CustomerId' give no insight to wether a customer may leave the bank. Rows with missing 'Tenure' values are filled with 0 as the rest of the data is valuable for it's insights into customer behavior.

### Encoding

In [ ]:
#Assigning to 'data_ohe' the dataframe encoded using One-Hot Encoding.
data_ohe = pd.get_dummies(data, drop_first=True)

In [ ]:
#Printing 'data_ohe'.
data_ohe

### Splitting Data

In [ ]:
# Assigning to 'target' the 'Exited' column of 'data_ohe'.
target = data_ohe['Exited']

# Assigning to 'features' all other columns but 'Exited' from 'data_ohe'.
features = data_ohe.drop('Exited', axis=1)

# Splitting data into train, validation, and test sets
features_train, features_temp, target_train, target_temp = train_test_split(
    features, target, test_size=0.4, random_state=12345
)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_temp, target_temp, test_size=0.5, random_state=12345
)

### Scaling

In [ ]:
#List of numerical features in the dataset.
numeric = ['Age', 'CreditScore', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary','Geography_Germany','Geography_Spain', 'Gender_Male']

In [ ]:
#Initializing StandardScaler to standardize numerical features.
scaler = StandardScaler()

#Fiting the scaler to the training set's numerical features.
scaler.fit(features_train[numeric])

#Transforming the numerical features of the training set using the fitted scaler.
features_train[numeric] = scaler.transform(features_train[numeric])

#Transforming the numerical features of the validation set using the same scaler.
features_valid[numeric] = scaler.transform(features_valid[numeric])

#Transforming the numerical features of the test set using the same scaler.
features_test[numeric] = scaler.transform(features_test[numeric])

#Printing the first few rows of the transformed training set to inspect the changes.
print(features_train.head())

In [ ]:
features_valid

In [ ]:
features_test

Because all features should be considered equally important before the algorithm's execution, I have standardized the data using StandardScaler.

## Model Exploration

In [ ]:
# Defining a custom scoring function using F1 score.
scorer = make_scorer(f1_score)

### Decision Tree Classifier

In [ ]:
#Defining the hyperparameters grid.
param_grid = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'criterion': ['gini', 'entropy']
}

#Initializing DecisionTreeClassifier model.
dtc_model = DecisionTreeClassifier(random_state=12345)

#Searching for the best combination of hyperparameters.
grid_search = GridSearchCV(dtc_model, param_grid, cv=5, scoring=scorer)
grid_search.fit(features_train, target_train)

#Assigning to 'best_dtc_params' the best hyperparameters.
best_dtc_params = grid_search.best_params_

#Training Model using 'features_train' and 'target train'.
best_dtc_model = DecisionTreeClassifier(**best_dtc_params, random_state=12345)
best_dtc_model.fit(features_train, target_train)

#Assigning to 'train_accuracy' score based on training values.
train_accuracy = best_dtc_model.score(features_train, target_train)

#Assigning to 'train_accuracy' score based on validation values.
valid_accuracy = best_dtc_model.score(features_valid, target_valid)

#Assigning to 'predicted_valid' model predictions using 'features_valid'.
predicted_valid = best_dtc_model.predict(features_valid)

#Printing the best hyperparameters found by grid search.
print("Best hyperparameters:", best_dtc_params)

#Printing the accuracy on the training set.
print("Accuracy on the training set:", train_accuracy)

#Printing the accuracy on the validation set.
print("Accuracy on the validation set:", valid_accuracy)

#Printing the F1 score for the validation set predictions.
print('F1 Score:', f1_score(target_valid, predicted_valid))

#Printing the recall score for the validation set predictions.
print('Recall Score:', recall_score(target_valid, predicted_valid))

#Printing the precision score for the validation set predictions.
print('Precision Score:', precision_score(target_valid, predicted_valid))

#Calculating the probabilities of class 1 for the validation set.
probabilities_valid = best_dtc_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

#Computing the false positive rate, true positive rate, and thresholds for the ROC curve.
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)

#Compingting the area under the ROC curve (AUC-ROC).
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc)

#Ploting the ROC curve.
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

The Decision Tree Classifier model shows decent performance, with relatively high accuracy and AUC-ROC. However, there is room for improvement, especially in terms of recall and F1 score, indicating that the model could better identify positive instances.

### Logistic Regression

In [ ]:
#Defining the hyperparameters grid.
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1.0, 10.0],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 300]
}

#Initializing LogisticRegression model.
lr_model = LogisticRegression(random_state=12345)

#Searching for the best combination of hyperparameters.
grid_search = GridSearchCV(lr_model, param_grid, cv=5, scoring=scorer)
grid_search.fit(features_train, target_train)

#Assigning to 'best_params' the best hyperparameters.
best_lr_params = grid_search.best_params_

#Training Model using 'features_train' and 'target train'.
best_lr_model = LogisticRegression(**best_lr_params, random_state=12345)
best_lr_model.fit(features_train, target_train)

#Assigning to 'train_accuracy' score based on training values.
train_accuracy = best_lr_model.score(features_train, target_train)

#Assigning to 'train_accuracy' score based on validation values.
valid_accuracy = best_lr_model.score(features_valid, target_valid)

#Assigning to 'predicted_valid' model predictions using 'features_valid'.
predicted_valid = best_lr_model.predict(features_valid)

#Printing findings.
print("Best hyperparameters:", best_lr_params)
print("Accuracy on the training set:", train_accuracy)
print("Accuracy on the validation set:", valid_accuracy)
print('F1 Score:', f1_score(target_valid, predicted_valid))
print('Recall Score:', recall_score(target_valid, predicted_valid))
print('Precision Score:', precision_score(target_valid, predicted_valid))

probabilities_valid = best_lr_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', auc_roc)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

The Logistic Regression model demonstrates moderate performance, with accuracy and AUC-ROC values indicating some ability to discriminate between classes. However, the F1 score and recall are relatively low, suggesting that the model struggles to identify positive instances effectively. There may be room for improvement through further tuning or exploration of different algorithms.

### Random Forest Classifier

In [ ]:
#Defining the hyperparameters grid.
param_grid = {
    'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'max_depth': [None, 10, 20, 30, 40, 50]
}

#Initializing RandomForestClassifier model.
rfc_model = RandomForestClassifier(random_state=12345)

#Searching for the best combination of hyperparameters.
grid_search = GridSearchCV(rfc_model, param_grid, cv=7, scoring=scorer)
grid_search.fit(features_train, target_train)

#Assigning to 'best_params' the best hyperparameters.
best_rfc_params = grid_search.best_params_

#Training Model using 'features_train' and 'target train'.
best_rfc_model = RandomForestClassifier(**best_rfc_params, random_state=12345)
best_rfc_model.fit(features_train, target_train)

#Assigning to 'train_accuracy' score based on training values.
train_accuracy = best_rfc_model.score(features_train, target_train)

#Assigning to 'train_accuracy' score based on validation values.
valid_accuracy = best_rfc_model.score(features_valid, target_valid)

#Assigning to 'predicted_valid' model predictions using 'features_valid'.
predicted_valid = best_rfc_model.predict(features_valid)

#Printing findings.
print("Best hyperparameters:", best_rfc_params)
print("Accuracy on the training set:", train_accuracy)
print("Accuracy on the validation set:", valid_accuracy)
print('F1 Score:', f1_score(target_valid, predicted_valid))
print('Recall Score:', recall_score(target_valid, predicted_valid))
print('Precision Score:', precision_score(target_valid, predicted_valid))

probabilities_valid = best_rfc_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', auc_roc)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

The Random Forest Classifier model demonstrates strong performance, with high accuracy and AUC-ROC values indicating good discrimination between classes. The F1 score and recall are relatively balanced, suggesting effective identification of positive instances. This model appears to be well-tuned and performs well on both training and validation datasets.

### Weight Balancing

In [1]:
#Defining the hyperparameters grid.
param_grid = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'criterion': ['gini', 'entropy'],
    'class_weight': ['balanced']
}

#Initializing DecisionTreeClassifier model.
dtc_model = DecisionTreeClassifier(random_state=12345)

#Searching for the best combination of hyperparameters.
grid_search = GridSearchCV(dtc_model, param_grid, cv=5, scoring=scorer)
grid_search.fit(features_train, target_train)

#Assigning to 'best_dtc_params' the best hyperparameters.
best_dtc_params = grid_search.best_params_

#Training Model using 'features_train' and 'target train'.
best_dtc_model = DecisionTreeClassifier(**best_dtc_params, random_state=12345)
best_dtc_model.fit(features_train, target_train)

#Assigning to 'train_accuracy' score based on training values.
train_accuracy = best_dtc_model.score(features_train, target_train)

#Assigning to 'train_accuracy' score based on validation values.
valid_accuracy = best_dtc_model.score(features_valid, target_valid)

#Assigning to 'predicted_valid' model predictions using 'features_valid'.
predicted_valid = best_dtc_model.predict(features_valid)

#Printing findings.
print("Best hyperparameters:", best_dtc_params)
print("Accuracy on the training set:", train_accuracy)
print("Accuracy on the validation set:", valid_accuracy)
print('F1 Score:', f1_score(target_valid, predicted_valid))
print('Recall Score:', recall_score(target_valid, predicted_valid))
print('Precision Score:', precision_score(target_valid, predicted_valid))

probabilities_valid = best_dtc_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', auc_roc)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

NameError: name 'DecisionTreeClassifier' is not defined

The DecisionTreeClassifier model using 'balanced' for class_weight demonstrates reasonable performance, with a balanced F1 score and relatively high recall, suggesting effective identification of positive instances. However, the precision score is comparatively lower, indicating a higher rate of false positive predictions. The AUC-ROC value suggests good discrimination between classes. This model may benefit from further optimization or exploration of different algorithms to improve precision while maintaining high recall.

In [ ]:
#Defining the hyperparameters grid.
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1.0, 10.0],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 300],
    'class_weight': ['balanced']
}

#Initializing LogisticRegression model.
lr_model = LogisticRegression(random_state=12345)

#Searching for the best combination of hyperparameters.
grid_search = GridSearchCV(lr_model, param_grid, cv=5, scoring=scorer)
grid_search.fit(features_train, target_train)

#Assigning to 'best_params' the best hyperparameters.
best_lr_params = grid_search.best_params_

#Training Model using 'features_train' and 'target train'.
best_lr_model = LogisticRegression(**best_lr_params, random_state=12345)
best_lr_model.fit(features_train, target_train)

#Assigning to 'train_accuracy' score based on training values.
train_accuracy = best_lr_model.score(features_train, target_train)

#Assigning to 'train_accuracy' score based on validation values.
valid_accuracy = best_lr_model.score(features_valid, target_valid)

#Assigning to 'predicted_valid' model predictions using 'features_valid'.
predicted_valid = best_lr_model.predict(features_valid)

#Printing findings.
print("Best hyperparameters:", best_lr_params)
print("Accuracy on the training set:", train_accuracy)
print("Accuracy on the validation set:", valid_accuracy)
print('F1 Score:', f1_score(target_valid, predicted_valid))
print('Recall Score:', recall_score(target_valid, predicted_valid))
print('Precision Score:', precision_score(target_valid, predicted_valid))

probabilities_valid = best_lr_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', auc_roc)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

The LogisticRegression model using 'balanced' class weight demonstrates moderate performance, with relatively low accuracy and precision scores. The recall score is higher, indicating a better ability to identify positive instances, but the precision is comparatively lower, suggesting a higher rate of false positive predictions. The AUC-ROC value suggests moderate discrimination between classes. This model may benefit from further optimization or exploration of different algorithms to improve overall performance.

In [ ]:
#Defining the hyperparameters grid.
param_grid = {
    'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'class_weight': ['balanced']
}

#Initializing RandomForestClassifier model.
rfc_model = RandomForestClassifier(random_state=12345)

#Searching for the best combination of hyperparameters.
grid_search = GridSearchCV(rfc_model, param_grid, cv=7, scoring=scorer)
grid_search.fit(features_train, target_train)

#Assigning to 'best_params' the best hyperparameters.
best_rfc_params = grid_search.best_params_

#Training Model using 'features_train' and 'target train'.
best_rfc_model = RandomForestClassifier(**best_rfc_params, random_state=12345)
best_rfc_model.fit(features_train, target_train)

#Assigning to 'train_accuracy' score based on training values.
train_accuracy = best_rfc_model.score(features_train, target_train)

#Assigning to 'train_accuracy' score based on validation values.
valid_accuracy = best_rfc_model.score(features_valid, target_valid)

#Assigning to 'predicted_valid' model predictions using 'features_valid'.
predicted_valid = best_rfc_model.predict(features_valid)

#Printing findings.
print("Best hyperparameters:", best_rfc_params)
print("Accuracy on the training set:", train_accuracy)
print("Accuracy on the validation set:", valid_accuracy)
print('F1 Score:', f1_score(target_valid, predicted_valid))
print('Recall Score:', recall_score(target_valid, predicted_valid))
print('Precision Score:', precision_score(target_valid, predicted_valid))

probabilities_valid = best_rfc_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', auc_roc)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

The RandomForestClassifier model using 'class_weight': 'balanced' demonstrates strong performance, with high accuracy and AUC-ROC values indicating good discrimination between classes. The F1 score and recall are relatively balanced, suggesting effective identification of positive instances. This model appears to be well-tuned and performs well on both training and validation datasets.

### Upsampling

In [ ]:
#Creating upsampling function
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )

    return features_upsampled, target_upsampled

In [ ]:
#Initializing variables to store the best F1 score and corresponding repeat value.
best_f1 = 0
best_repeat = 0

#Iterating over a range of values (0 to 19) for the repeat parameter.
for i in range(20):
    
    #Upsampling the training data using the current value of the repeat parameter.
    features_upsampled, target_upsampled = upsample(features_train, target_train, i)

    #Initializing a RandomForestClassifier model with the best hyperparameters.
    best_rfc_model = RandomForestClassifier(**best_rfc_params, random_state=12345)
    
    #Training the RandomForestClassifier model on the upsampled data.
    best_rfc_model.fit(features_upsampled, target_upsampled)

    #Predicting the target variable for the validation set using the trained model.
    predicted_valid = best_rfc_model.predict(features_valid)
    
    #Calculating the F1 score for the predictions on the validation set.
    f1 = f1_score(target_valid, predicted_valid)
    
    #Updating the best F1 score and corresponding repeat value if the current F1 score is higher.
    if f1 > best_f1:
        best_f1 = f1
        best_repeat = i

#Printing the best repeat value and corresponding F1 score
print('Best repeat n:', best_repeat)
print('Best F1:', best_f1)

### Downsampling

In [ ]:
def downsample(features, target, fraction):
    #Separating features and target for each class (0 and 1).
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    #Downsampling the majority class (class 0) to match the specified fraction.
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)]  # Sample fraction of class 0.
        + [features_ones]  #Combining with all instances of class 1
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)]  # Sample fraction of class 0.
        + [target_ones]  #Combining with all instances of class 1.
    )

    #Shuffling the downsampled data to randomize the order.
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345
    )

    return features_downsampled, target_downsampled

In [ ]:
#Downsampling the majority class in the training set to achieve a balanced dataset.
features_downsampled, target_downsampled = downsample(features_train, target_train, 1)

#Initializing a RandomForestClassifier model with the best hyperparameters.
best_rfc_model = RandomForestClassifier(**best_rfc_params, random_state=12345)

#Training the RandomForestClassifier model on the downsampled training data.
best_rfc_model.fit(features_downsampled, target_downsampled)

#Predicting the target variable for the validation set using the trained model.
predicted_valid = best_rfc_model.predict(features_valid)

#Calculating the F1 score for the predictions on the validation set.
print('F1:', f1_score(target_valid, predicted_valid))

## Final Model

In [ ]:
#Upsampling the minority class in the training set to achieve a balanced dataset.
features_upsampled, target_upsampled = upsample(features_train, target_train, 8)

#Training Model using 'features_upsampled' and 'target_upsampled train'.
best_rfc_model = RandomForestClassifier(**best_rfc_params, random_state=12345)
best_rfc_model.fit(features_upsampled, target_upsampled)

#Assigning to 'train_accuracy' score based on training values.
train_accuracy = best_rfc_model.score(features_train, target_train)

#Assigning to 'train_accuracy' score based on validation values.
valid_accuracy = best_rfc_model.score(features_valid, target_valid)

#Assigning to 'predicted_valid' model predictions using 'features_valid'.
predicted_valid = best_rfc_model.predict(features_valid)

#Printing findings.
print("Best hyperparameters:", best_rfc_params)
print("Accuracy on the training set:", train_accuracy)
print("Accuracy on the validation set:", valid_accuracy)
print('F1 Score:', f1_score(target_valid, predicted_valid))
print('Recall Score:', recall_score(target_valid, predicted_valid))
print('Precision Score:', precision_score(target_valid, predicted_valid))

probabilities_valid = best_rfc_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC:', auc_roc)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

In [ ]:
#Assigning to 'test_accuracy' score based on test values.
test_accuracy = best_rfc_model.score(features_test, target_test)

#Assigning to 'predicted_test' model predictions using 'features_test'.
predicted_test = best_rfc_model.predict(features_test)

#Printing findings.
print("Accuracy on the training set:", train_accuracy)
print("Accuracy on the test set:", test_accuracy)
print('F1 Score:', f1_score(target_test, predicted_test))
print('Recall Score:', recall_score(target_test, predicted_test))
print('Precision Score:', precision_score(target_test, predicted_test))

probabilities_test = best_rfc_model.predict_proba(features_test)
probabilities_one_test = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_test)
auc_roc = roc_auc_score(target_test, probabilities_one_test)

print('AUC-ROC:', auc_roc)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

## Conclusion

The final and most effective model was the RandomForestClassifier using balanced as the class weight, a max depth of 10 and 13 estimators using upsampled data, and produced a balanced F1 score and relatively high recall, indicating effective identification of positive instances. The precision score is moderate, suggesting a reasonable proportion of true positive predictions among all positive predictions made by the model. The AUC-ROC value indicates good discrimination between classes.